In [ ]:
from collections import defaultdict

import matplotlib.pyplot as plt
from pyodesys.symbolic import symmetricsys, get_logexp
from pyodesys.native import native_sys
from _chem_kinet import from_b64_gz_pkl, radiolysis1
%matplotlib inline

In [ ]:
linsys, (params,) = from_b64_gz_pkl(radiolysis1, steady_state_root=True)

In [ ]:
def integrate_and_plot(odesys, tend=1e6, **kwargs):
    res = odesys.integrate(tend, defaultdict(
            float, {'H2O': 55.4e3, 'H+': 1e-4, 'OH-': 1e-4, 'O2': 1e-6, 'H2O2': [0, 1e-4, 1e-3]}
        ), params, integrator='cvode', **kwargs)
    fig, axes = plt.subplots(2, len(res), figsize=(16, 10))
    plot_kw = dict(legend=dict(loc='center left', prop={'size': 8}), xscale='log')
    for r, axcol in zip(res, axes.T):
        r.plot(ax=axcol[0], yscale='log', **plot_kw)
        r.plot(ax=axcol[1], deriv=True, yscale='symlog;linthreshy=1e-9', **plot_kw);
        for ax in axcol:
            ax.set_xlim([1e-13, 1e-3])
            ax.set_xlim([1e-6, tend])

In [ ]:
integrate_and_plot(linsys, nsteps=5000, atol=1e-13, rtol=1e-13)

In [ ]:
integrate_and_plot(linsys, return_on_root=True, nsteps=5000, atol=1e-13, rtol=1e-13)

In [ ]:
native = native_sys['cvode'].from_other(linsys)
print(next(filter(lambda s: s.endswith('.cpp'), native._native._written_files)))

In [ ]:
integrate_and_plot(native, nsteps=5000, atol=1e-13, rtol=1e-13, return_on_root=True)

In [ ]:
LogLogSys = symmetricsys(get_logexp(1, 1e-24), get_logexp(1, 1e-12))
tsys = LogLogSys.from_other(linsys)

In [ ]:
# would neeed to tile in pre_process:
#integrate_and_plot(tsys)